# Chapter 14: Kubernetes Core Resources

Having established your development environment and mastered basic kubectl operations in Chapter 13, we now examine the fundamental building blocks of Kubernetes applications. This chapter explores the core API resources that transform container images from your registry (Chapter 11) into running, networked, and managed workloads orchestrated by the control plane (Chapter 12).

These resources form the declarative vocabulary through which you communicate application requirements to Kubernetes. Understanding their nuances—from Pod lifecycle hooks to Deployment strategy nuances—separates operational Kubernetes proficiency from basic container running.

## 14.1 Pods

The Pod is the smallest deployable unit in Kubernetes. Unlike Docker containers which run individually, Kubernetes wraps containers into Pods, providing shared execution contexts including networking, storage, and metadata.

### Pod Fundamentals

A Pod represents a single instance of a running process in your cluster. It can contain one or more containers that share:
- **Network namespace**: Single IP address, shared localhost interface
- **Storage volumes**: Shared filesystem mounts
- **IPC namespace**: Inter-process communication capabilities
- **UTS namespace**: Shared hostname

**Single vs. Multi-Container Pods:**
While Pods can run multiple containers, adhere to the "one main process per container" philosophy. Use multi-container Pods only for tightly coupled auxiliary processes (sidecars, adapters, proxies) that must share the same lifecycle and filesystem.

```yaml
apiVersion: v1
kind: Pod
metadata:
  name: single-container-pod
  namespace: default
  labels:
    app: web
    version: v1.0
spec:
  containers:
  - name: nginx
    image: nginx:1.25-alpine
    ports:
    - containerPort: 80
      protocol: TCP
    resources:
      requests:
        memory: "64Mi"
        cpu: "100m"
      limits:
        memory: "128Mi"
        cpu: "200m"
```

### 14.1.1 Pod Lifecycle

Pods follow a defined lifecycle from creation through termination, progressing through distinct phases.

**Pod Phases:**
- **Pending**: Accepted by cluster, container images not yet pulled/started
- **Running**: At least one container executing (or completed for restartPolicy: Never)
- **Succeeded**: All containers terminated successfully (exit 0), not restarting
- **Failed**: All containers terminated, at least one failed (non-zero exit)
- **Unknown**: State cannot be determined (typically node communication lost)

**Container States:**
Within each Pod, individual containers maintain states:
- **Waiting**: Image pulling, volume mounting, or waiting for dependencies
- **Running**: Executing without issues
- **Terminated**: Completed execution or failed, includes exit code and reason

**Lifecycle Hooks:**
Execute commands at specific lifecycle events:

```yaml
apiVersion: v1
kind: Pod
spec:
  containers:
  - name: lifecycle-demo
    image: nginx:alpine
    lifecycle:
      postStart:
        exec:
          command: ["/bin/sh", "-c", "echo 'Container started' >> /var/log/nginx/start.log"]
      preStop:
        exec:
          command: ["/bin/sh", "-c", "nginx -s quit; sleep 10"]  # Graceful shutdown
```

**Restart Policies:**
Control behavior when containers fail:
- **Always** (default): Restart regardless of exit code (suitable for long-running services)
- **OnFailure**: Restart only on non-zero exit codes (suitable for batch jobs)
- **Never**: Do not restart; Pod remains in Failed phase (suitable for one-time tasks)

```yaml
spec:
  restartPolicy: OnFailure  # For batch workloads
```

### 14.1.2 Pod Specifications

The PodSpec defines runtime behavior, resource allocation, and security contexts.

**Security Contexts:**
Run containers with specific security constraints:

```yaml
spec:
  securityContext:
    runAsNonRoot: true
    runAsUser: 1000
    runAsGroup: 3000
    fsGroup: 2000
    seccompProfile:
      type: RuntimeDefault
  containers:
  - name: secure-app
    image: myapp:latest
    securityContext:
      allowPrivilegeEscalation: false
      readOnlyRootFilesystem: true
      capabilities:
        drop:
        - ALL
        add:
        - NET_BIND_SERVICE  # Only if binding low ports required
```

**DNS Policy:**
Control DNS resolution behavior:

```yaml
spec:
  dnsPolicy: ClusterFirst  # Default: cluster DNS for cluster domains, upstream otherwise
  # Alternative: ClusterFirstWithHostNet (for host networking)
  # Alternative: Default (inherit from node)
  dnsConfig:
    nameservers:
      - 8.8.8.8
    searches:
      - mycompany.local
    options:
      - name: ndots
        value: "2"
```

**Scheduling Constraints:**
Direct Pod placement:

```yaml
spec:
  nodeSelector:
    disktype: ssd
  affinity:
    nodeAffinity:
      requiredDuringSchedulingIgnoredDuringExecution:
        nodeSelectorTerms:
        - matchExpressions:
          - key: kubernetes.io/arch
            operator: In
            values: ["amd64"]
    podAntiAffinity:
      preferredDuringSchedulingIgnoredDuringExecution:
      - weight: 100
        podAffinityTerm:
          labelSelector:
            matchExpressions:
            - key: app
              operator: In
              values: ["web"]
          topologyKey: kubernetes.io/hostname
  tolerations:
  - key: "dedicated"
    operator: "Equal"
    value: "web"
    effect: "NoSchedule"
```

### 14.1.3 Init Containers

Init containers run before application containers start, completing initialization tasks such as database migrations, configuration generation, or permission fixes.

**Characteristics:**
- Run sequentially in defined order
- Must complete successfully (exit 0) before main containers start
- Have separate resource limits from application containers
- Do not support lifecycle hooks, liveness/readiness probes, or restart policies

```yaml
apiVersion: v1
kind: Pod
metadata:
  name: init-demo
spec:
  initContainers:
  - name: init-migrate
    image: myapp:latest
    command: ['python', 'manage.py', 'migrate']
    env:
    - name: DATABASE_URL
      valueFrom:
        secretKeyRef:
          name: db-credentials
          key: url
    volumeMounts:
    - name: data
      mountPath: /data
  - name: init-permissions
    image: busybox:1.36
    command: ['sh', '-c', 'chmod 700 /data && chown 1000:1000 /data']
    volumeMounts:
    - name: data
      mountPath: /data
  containers:
  - name: app
    image: myapp:latest
    ports:
    - containerPort: 8080
    volumeMounts:
    - name: data
      mountPath: /data
  volumes:
  - name: data
    emptyDir: {}
```

**Sidecar Pattern:**
While init containers handle initialization, sidecars (additional containers in the Pod) handle ongoing auxiliary functions:

```yaml
spec:
  containers:
  - name: main-app
    image: myapp:latest
  - name: nginx-sidecar
    image: nginx:alpine
    volumeMounts:
    - name: shared-logs
      mountPath: /var/log/nginx
  - name: log-aggregator
    image: fluentd:latest
    volumeMounts:
    - name: shared-logs
      mountPath: /var/log/app
  volumes:
  - name: shared-logs
    emptyDir: {}
```

### 14.1.4 Resource Requests and Limits

Resource specifications ensure fair scheduling and prevent noisy neighbor problems.

**Resource Types:**
- **CPU**: Measured in millicores (m) or cores. `100m` = 0.1 CPU cores
- **Memory**: Measured in bytes (Ki, Mi, Gi) or raw bytes
- **Ephemeral Storage**: Container writable layer and emptyDir volumes (Kubernetes 1.21+)

**Best Practices:**

```yaml
spec:
  containers:
  - name: web
    image: nginx:alpine
    resources:
      requests:
        memory: "128Mi"    # Guaranteed minimum; used for scheduling
        cpu: "100m"        # 0.1 cores; used for scheduling
        ephemeral-storage: "1Gi"
      limits:
        memory: "256Mi"    # Hard limit; OOMKill if exceeded
        cpu: "500m"        # Throttled if exceeded, not killed
        ephemeral-storage: "2Gi"
```

**Quality of Service (QoS) Classes:**
Kubernetes assigns QoS classes based on resource specifications:
- **Guaranteed**: Requests = Limits for all resources (highest priority, least likely evicted)
- **Burstable**: Requests < Limits for at least one resource (default for specified resources)
- **BestEffort**: No requests or limits specified (lowest priority, first evicted)

**Vertical Pod Autoscaler (VPA) Recommendation:**
For initial sizing, use VPA in recommendation mode:
```yaml
apiVersion: autoscaling.k8s.io/v1
kind: VerticalPodAutoscaler
metadata:
  name: web-vpa
spec:
  targetRef:
    apiVersion: apps/v1
    kind: Deployment
    name: web
  updatePolicy:
    updateMode: "Off"  # Recommend only, don't auto-update
```

## 14.2 Deployments

Deployments provide declarative updates for Pods and ReplicaSets, managing the lifecycle of stateless applications. They handle rolling updates, rollback capabilities, and scaling operations.

### 14.2.1 ReplicaSets

A ReplicaSet ensures a specified number of Pod replicas run at any given time. While you rarely create ReplicaSets directly (Deployments manage them), understanding their function clarifies Deployment behavior.

**ReplicaSet Function:**
- Maintains stable set of replica Pods
- Uses label selectors to identify managed Pods
- Creates/deletes Pods to match desired replica count
- Supports equality-based selectors (not set-based)

```yaml
apiVersion: apps/v1
kind: ReplicaSet
metadata:
  name: frontend
  labels:
    app: guestbook
    tier: frontend
spec:
  replicas: 3
  selector:
    matchLabels:
      tier: frontend
    matchExpressions:
      - {key: tier, operator: In, values: [frontend]}
  template:
    metadata:
      labels:
        tier: frontend
    spec:
      containers:
      - name: php-redis
        image: gcr.io/google_samples/gb-frontend:v3
```

**Warning:** Do not manage ReplicaSets directly. Use Deployments which manage ReplicaSets, providing update strategies and rollback capabilities.

### 14.2.2 Rolling Updates

Deployments manage application updates through configurable strategies that replace old Pods with new ones gradually.

**RollingUpdate Strategy:**
The default strategy replaces Pods incrementally to ensure zero downtime.

```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: nginx-deployment
  labels:
    app: nginx
spec:
  replicas: 10
  strategy:
    type: RollingUpdate
    rollingUpdate:
      maxSurge: 25%        # Maximum pods above desired count during update (absolute number or %)
      maxUnavailable: 25%  # Maximum pods unavailable during update
  selector:
    matchLabels:
      app: nginx
  template:
    metadata:
      labels:
        app: nginx
    spec:
      containers:
      - name: nginx
        image: nginx:1.25
        ports:
        - containerPort: 80
```

**Update Mechanics:**
1. Deployment creates new ReplicaSet with updated Pod template
2. Scales up new ReplicaSet by `maxSurge` while scaling down old by `maxUnavailable`
3. Continues until all replicas run new version
4. Old ReplicaSet retained (replicas=0) for rollback capability

**Monitoring Updates:**
```bash
# Watch rollout status
kubectl rollout status deployment/nginx-deployment

# View ReplicaSets (shows old and new)
kubectl get rs -l app=nginx

# View rollout history
kubectl rollout history deployment/nginx-deployment
```

### 14.2.3 Rollbacks

Deployments retain revision history, enabling reversion to previous stable states when updates fail.

**Rollback Commands:**
```bash
# Check revision history
kubectl rollout history deployment/nginx-deployment

# Rollback to previous version
kubectl rollout undo deployment/nginx-deployment

# Rollback to specific revision
kubectl rollout undo deployment/nginx-deployment --to-revision=2

# Pause rollout for investigation
kubectl rollout pause deployment/nginx-deployment

# Resume rollout
kubectl rollout resume deployment/nginx-deployment
```

**Revision History Limits:**
Control history retention to manage etcd storage:

```yaml
spec:
  revisionHistoryLimit: 10  # Default is 10; old ReplicaSets cleaned up beyond this
```

### 14.2.4 Deployment Strategies

Beyond RollingUpdate, Kubernetes supports Recreate strategy and custom progressive delivery patterns.

**Recreate Strategy:**
Terminate all old Pods before creating new ones (causes downtime but avoids running two versions simultaneously).

```yaml
spec:
  strategy:
    type: Recreate  # Use when application cannot tolerate mixed versions
```

**Blue/Green Deployment (Manual):**
Maintain two environments, switching service selectors:

```yaml
# Blue deployment (current)
apiVersion: apps/v1
kind: Deployment
metadata:
  name: app-blue
spec:
  replicas: 3
  selector:
    matchLabels:
      app: myapp
      version: blue
  template:
    metadata:
      labels:
        app: myapp
        version: blue
    spec:
      containers:
      - name: app
        image: myapp:v1

---
# Service pointing to blue
apiVersion: v1
kind: Service
metadata:
  name: myapp
spec:
  selector:
    app: myapp
    version: blue  # Switch to green when ready
  ports:
  - port: 80
```

**Canary Deployment (Advanced):**
Use separate Deployments with traffic splitting (requires Service Mesh or Ingress controller):

```yaml
# Primary (90% traffic)
apiVersion: apps/v1
kind: Deployment
metadata:
  name: app-primary
spec:
  replicas: 9
  selector:
    matchLabels:
      app: myapp
      track: stable
---
# Canary (10% traffic)
apiVersion: apps/v1
kind: Deployment
metadata:
  name: app-canary
spec:
  replicas: 1
  selector:
    matchLabels:
      app: myapp
      track: canary
```

## 14.3 Services

Services provide stable networking endpoints for Pods, abstracting away the ephemeral nature of individual containers. Since Pods are created and destroyed dynamically, their IP addresses change; Services provide a constant IP and DNS name that routes to healthy Pods.

### Service Types Overview

Kubernetes supports four primary Service types, each suited to different exposure requirements.

### 14.3.1 ClusterIP

ClusterIP is the default Service type, exposing the Service on an internal IP accessible only within the cluster. This is ideal for internal microservice communication.

```yaml
apiVersion: v1
kind: Service
metadata:
  name: backend-api
  namespace: production
spec:
  type: ClusterIP
  selector:
    app: backend
    tier: api
  ports:
  - protocol: TCP
    port: 8080        # Service port exposed within cluster
    targetPort: 8080  # Container port to forward to
    name: http
  - protocol: TCP
    port: 8443
    targetPort: 8443
    name: https
  sessionAffinity: None  # or ClientIP for sticky sessions
```

**DNS Resolution:**
ClusterIP Services receive DNS entries automatically:
- Short name: `backend-api` (same namespace)
- FQDN: `backend-api.production.svc.cluster.local`

**Headless Services (Preliminary):**
When clusterIP: None, DNS returns Pod IPs directly (useful for StatefulSets):
```yaml
spec:
  clusterIP: None  # Headless
  selector:
    app: db
```

### 14.3.2 NodePort

NodePort exposes the Service on each Node's IP at a static port (30000-32767 by default), allowing external access without a cloud load balancer.

```yaml
apiVersion: v1
kind: Service
metadata:
  name: web-nodeport
spec:
  type: NodePort
  selector:
    app: web
  ports:
  - port: 80
    targetPort: 8080
    nodePort: 30080  # Optional: auto-allocated if unspecified
    name: http
```

**Access Pattern:**
`curl http://<Node-IP>:30080`

**Considerations:**
- Port range limited (30000-32767 by default; configurable via `--service-node-port-range`)
- Exposes nodes directly; security groups must allow NodePort range
- Traffic routes through kube-proxy (iptables/IPVS) adding slight latency
- Not recommended for production external exposure; use LoadBalancer or Ingress instead

### 14.3.3 LoadBalancer

LoadBalancer exposes the Service externally using a cloud provider's load balancer (AWS ELB, Azure Load Balancer, GCP Forwarding Rule).

```yaml
apiVersion: v1
kind: Service
metadata:
  name: web-lb
  annotations:
    service.beta.kubernetes.io/aws-load-balancer-type: "nlb"  # Network LB
    service.beta.kubernetes.io/aws-load-balancer-scheme: "internet-facing"
spec:
  type: LoadBalancer
  selector:
    app: web
  ports:
  - port: 80
    targetPort: 8080
  externalTrafficPolicy: Local  # Preserve client source IP; no kube-proxy hop
  healthCheckNodePort: 30000     # Required when externalTrafficPolicy=Local
```

**External Traffic Policy:**
- **Cluster** (default): Routes to any node, then kube-proxy forwards to Pod (may cross nodes, masks client IP)
- **Local**: Routes only to nodes running Service endpoints (preserves client IP, no extra hop, but uneven distribution if node pods differ)

**MetalLB (On-Premises):**
For bare-metal clusters without cloud providers, MetalLB provides LoadBalancer functionality:
```yaml
apiVersion: metallb.io/v1beta1
kind: IPAddressPool
metadata:
  name: production
spec:
  addresses:
  - 192.168.1.240-192.168.1.250
---
apiVersion: metallb.io/v1beta1
kind: L2Advertisement
metadata:
  name: production
spec:
  ipAddressPools:
  - production
```

### 14.3.4 Headless Services

Headless Services (clusterIP: None) do not allocate a virtual IP. Instead, they return the IPs of the backing Pods directly via DNS, useful for direct Pod-to-Pod communication and StatefulSets.

**Use Cases:**
- StatefulSets requiring stable network identity (database clusters)
- Client-side load balancing when applications must discover individual Pods
- Direct Pod communication without kube-proxy intermediation

```yaml
apiVersion: v1
kind: Service
metadata:
  name: db-headless
spec:
  clusterIP: None  # Headless
  selector:
    app: db
  ports:
  - port: 5432
    name: postgres
```

**DNS Behavior:**
Headless Services return A records for each Pod IP:
```bash
nslookup db-headless
# Returns:
# db-headless.default.svc.cluster.local.  IN A 10.244.1.5
# db-headless.default.svc.cluster.local.  IN A 10.244.2.8
```

**StatefulSet Integration:**
```yaml
apiVersion: apps/v1
kind: StatefulSet
metadata:
  name: postgres
spec:
  serviceName: "db-headless"  # Governs network identity
  replicas: 3
  selector:
    matchLabels:
      app: db
  template:
    metadata:
      labels:
        app: db
    spec:
      containers:
      - name: postgres
        image: postgres:15
```

**Pod Identity:**
StatefulSets with Headless Services create predictable DNS names:
- `postgres-0.db-headless.default.svc.cluster.local`
- `postgres-1.db-headless.default.svc.cluster.local`
- `postgres-2.db-headless.default.svc.cluster.local`

## 14.4 ConfigMaps

ConfigMaps decouple configuration artifacts from image content, allowing you to change application configuration without rebuilding containers. They store non-sensitive data such as config files, command-line arguments, or environment variables.

**Creation Methods:**

**From Literal Values:**
```bash
kubectl create configmap app-config \
  --from-literal=LOG_LEVEL=info \
  --from-literal=MAX_CONNECTIONS=100
```

**From Files:**
```bash
kubectl create configmap nginx-config \
  --from-file=nginx.conf=./configs/nginx.conf \
  --from-file=ssl.conf=./configs/ssl.conf
```

**From Directory:**
```bash
kubectl create configmap app-configs \
  --from-file=./configs/
```

**YAML Definition:**

```yaml
apiVersion: v1
kind: ConfigMap
metadata:
  name: app-config
  namespace: production
data:
  # Key-value pairs
  database.host: "postgres.production.svc.cluster.local"
  database.port: "5432"
  features.caching: "enabled"
  
  # File-like content
  app.properties: |
    log.level=INFO
    log.format=json
    max.threads=50
    
  # JSON configuration
  settings.json: |
    {
      "timeout": 30,
      "retries": 3,
      "backoff": "exponential"
    }
```

**Consumption in Pods:**

**Environment Variables:**
```yaml
spec:
  containers:
  - name: app
    image: myapp:latest
    env:
    # Single value
    - name: DATABASE_HOST
      valueFrom:
        configMapKeyRef:
          name: app-config
          key: database.host
    # All keys as env vars (prefix optional)
    envFrom:
    - configMapRef:
        name: app-config
        prefix: CONFIG_
```

**Volume Mounts:**
```yaml
spec:
  containers:
  - name: app
    image: nginx:alpine
    volumeMounts:
    - name: config-vol
      mountPath: /etc/nginx/conf.d
      readOnly: true
  volumes:
  - name: config-vol
    configMap:
      name: nginx-config
      items:  # Optional: select specific keys
      - key: nginx.conf
        path: default.conf
      - key: ssl.conf
        path: ssl.conf
      defaultMode: 0444  # File permissions
```

**Immutable ConfigMaps:**
Mark ConfigMaps as immutable to improve performance and prevent accidental changes:

```yaml
apiVersion: v1
kind: ConfigMap
metadata:
  name: immutable-config
immutable: true  # Cannot be updated without recreation
data:
  config.key: value
```

**Limitations:**
- Maximum size: 1MiB (etcd limit)
- No built-in versioning; changes propagate immediately to mounted volumes (kubelet syncs periodically)
- Not suitable for sensitive data (use Secrets instead)

## 14.5 Secrets

Secrets store sensitive data such as passwords, OAuth tokens, SSH keys, and TLS certificates. While similar to ConfigMaps, Secrets are base64 encoded (not encrypted by default) and designed for confidential data.

**Types of Secrets:**

**Opaque (Generic):**
Default type for arbitrary key-value data.

```yaml
apiVersion: v1
kind: Secret
metadata:
  name: db-credentials
type: Opaque
stringData:  # stringData automatically base64 encodes; prefer over data
  username: admin
  password: P@ssw0rd123!
  host: postgres.production.svc.cluster.local
data:  # base64 encoded manually if using data field
  legacy: YWRtaW4=  # echo -n 'admin' | base64
```

**TLS Certificates:**
```bash
kubectl create secret tls tls-cert \
  --cert=path/to/cert.crt \
  --key=path/to/key.key
```

**Docker Registry:**
```bash
kubectl create secret docker-registry regcred \
  --docker-server=registry.company.com \
  --docker-username=developer \
  --docker-password=token123 \
  --docker-email=dev@company.com
```

**Basic Authentication:**
```bash
kubectl create secret generic basic-auth \
  --from-literal=username=admin \
  --from-literal=password=secret
```

**Consumption:**

**Environment Variables (Risky):**
```yaml
spec:
  containers:
  - name: app
    image: myapp:latest
    env:
    - name: DB_PASSWORD
      valueFrom:
        secretKeyRef:
          name: db-credentials
          key: password
          optional: false  # Pod fails to start if secret missing
```

**Volume Mounts (Preferred for Security):**
Secrets mounted as volumes are stored in tmpfs (RAM), never touching node disk:

```yaml
spec:
  containers:
  - name: app
    image: myapp:latest
    volumeMounts:
    - name: secret-vol
      mountPath: /etc/secrets
      readOnly: true
  volumes:
  - name: secret-vol
    secret:
      secretName: db-credentials
      items:
      - key: password
        path: db-password.txt
        mode: 0400  # Restrictive permissions
```

**Security Considerations:**

1. **Encryption at Rest:**
   Enable etcd encryption for Secrets (disabled by default):
   ```yaml
   apiVersion: apiserver.config.k8s.io/v1
   kind: EncryptionConfiguration
   resources:
   - resources:
     - secrets
     providers:
     - aescbc:
         keys:
         - name: key1
           secret: <base64-encoded-32-byte-key>
     - identity: {}  # Fallback for unencrypted resources
   ```

2. **RBAC Restriction:**
   ```yaml
   apiVersion: rbac.authorization.k8s.io/v1
   kind: Role
   metadata:
     name: secret-reader
   rules:
   - apiGroups: [""]
     resources: ["secrets"]
     verbs: ["get", "list"]
     resourceNames: ["specific-secret"]  # Limit to specific secrets
   ```

3. **External Secret Operators:**
   For production, use external secret management (HashiCorp Vault, AWS Secrets Manager) with operators like External Secrets Operator or Vault Agent Injector rather than storing sensitive data in etcd.

## 14.6 Namespaces

While introduced in Chapter 13, Namespaces warrant deeper examination as resource objects themselves, particularly regarding resource quotas and limits.

**Resource Quotas:**
Limit aggregate resource consumption per namespace:

```yaml
apiVersion: v1
kind: ResourceQuota
metadata:
  name: compute-quota
  namespace: development
spec:
  hard:
    requests.cpu: "20"
    requests.memory: 40Gi
    limits.cpu: "40"
    limits.memory: 80Gi
    pods: "50"
    services: "20"
    persistentvolumeclaims: "10"
```

**Limit Ranges:**
Default and enforce container resource specifications:

```yaml
apiVersion: v1
kind: LimitRange
metadata:
  name: default-limits
  namespace: development
spec:
  limits:
  - default:
      cpu: "500m"
      memory: "512Mi"
    defaultRequest:
      cpu: "100m"
      memory: "128Mi"
    type: Container
  - max:
      cpu: "2"
      memory: 2Gi
    min:
      cpu: "50m"
      memory: "64Mi"
    type: Container
```

**Network Policies:**
Namespace-scoped firewall rules (requires CNI support):

```yaml
apiVersion: networking.k8s.io/v1
kind: NetworkPolicy
metadata:
  name: api-allow
  namespace: production
spec:
  podSelector:
    matchLabels:
      app: api
  policyTypes:
  - Ingress
  - Egress
  ingress:
  - from:
    - namespaceSelector:
        matchLabels:
          name: frontend
    - podSelector:
        matchLabels:
          app: web
    ports:
    - protocol: TCP
      port: 8080
  egress:
  - to:
    - namespaceSelector:
        matchLabels:
          name: database
    ports:
    - protocol: TCP
      port: 5432
```

## 14.7 Labels and Selectors

Labels are key-value pairs attached to objects for identification and grouping. Selectors query these labels to filter resources, forming the loose coupling mechanism between Services and Pods, or Deployments and ReplicaSets.

**Label Syntax:**
- Keys: Optional prefix (DNS subdomain) + name, up to 63 characters
- Values: Up to 63 characters, alphanumeric with `-`, `_`, `.`

**Common Label Conventions:**
```yaml
metadata:
  labels:
    app.kubernetes.io/name: mysql
    app.kubernetes.io/instance: wordpress-abcx
    app.kubernetes.io/version: "5.7.21"
    app.kubernetes.io/component: database
    app.kubernetes.io/part-of: wordpress
    app.kubernetes.io/managed-by: helm
    environment: production
    tier: backend
```

**Selector Types:**

**Equality-Based:**
```yaml
selector:
  matchLabels:
    app: nginx
    environment: production
```

**Set-Based (Supported in Deployments, Services use equality):**
```yaml
selector:
  matchExpressions:
  - key: tier
    operator: In
    values: [frontend, backend]
  - key: environment
    operator: NotIn
    values: [development]
  - key: version
    operator: Exists  # Key exists, any value
```

**CLI Usage:**
```bash
# Select by label
kubectl get pods -l app=nginx,environment=production

# Set-based in CLI
kubectl get pods -l 'environment notin (development,staging)'

# Remove label
kubectl label pods my-pod app-

# Add/overwrite label
kubectl label pods my-pod version=2.0 --overwrite
```

**Service Selector Matching:**
Services route traffic to Pods with matching labels, regardless of how Pods were created:
```yaml
# Service selects Pods with these labels
selector:
  app: web
  version: v2
```

## 14.8 Annotations

Unlike labels used for querying and selection, annotations store non-identifying metadata for external tooling, automation, or administrative purposes.

**Use Cases:**
- Build/release tracking (git commit SHA, build ID)
- Configuration for ingress controllers (rewrite targets, SSL redirects)
- Audit timestamps and owner information
- Phone/pager contact for on-call rotation
- Image policy requirements

**Syntax:**
```yaml
metadata:
  annotations:
    deployment.kubernetes.io/revision: "3"
    kubectl.kubernetes.io/last-applied-configuration: |
      {"apiVersion":"apps/v1","kind":"Deployment"...}
    prometheus.io/scrape: "true"
    prometheus.io/port: "8080"
    prometheus.io/path: "/metrics"
    nginx.ingress.kubernetes.io/rewrite-target: /
    nginx.ingress.kubernetes.io/ssl-redirect: "true"
    vault.hashicorp.com/agent-inject: "true"
    vault.hashicorp.com/role: "app"
    company.com/build-id: "20231102.3"
    company.com/owner: "platform-team@company.com"
```

**Size Limit:**
Annotations total 256KB per object (including system annotations).

**Accessing Annotations:**
```bash
# View annotations
kubectl get deployment myapp -o jsonpath='{.metadata.annotations}'

# Add annotation
kubectl annotate deployment myapp company.com/release-date="2023-11-02"

# Remove annotation
kubectl annotate deployment myapp company.com/release-date-
```

---

## Chapter Summary and Preview

In this chapter, we examined Kubernetes' fundamental resource types that transform container images into managed, networked applications. We explored Pods as the atomic deployment unit, detailing lifecycle phases, init containers for initialization sequences, and resource specifications ensuring cluster stability. Deployments emerged as the primary workload controller, managing ReplicaSets to maintain desired state while enabling sophisticated update strategies including rolling updates with configurable surge/unavailability parameters and instant rollback capabilities. Services provided stable networking abstractions across ClusterIP for internal communication, NodePort for direct node access, LoadBalancer for cloud integration, and Headless Services for direct Pod discovery. ConfigMaps and Secrets decoupled configuration from application code, with Secrets providing additional security considerations for sensitive data. Finally, Labels and Selectors established the loose coupling mechanism enabling Services to discover dynamic Pod sets, while Annotations stored metadata for tooling and automation.

**Key Takeaways:**
- Always use Deployments rather than managing Pods or ReplicaSets directly to gain self-healing, scaling, and update capabilities.
- Specify resource requests and limits for every container to ensure predictable scheduling and prevent resource starvation.
- Use ClusterIP Services for internal communication; expose via Ingress rather than NodePort for production external access.
- Mount Secrets as volumes rather than environment variables to prevent exposure in process lists and core dumps.
- Implement Immutable ConfigMaps when configuration rarely changes to improve performance and prevent drift.
- Label resources consistently using the recommended kubernetes.io prefixes for tooling compatibility.

**Next Chapter Preview:**
Chapter 15: Kubernetes Networking explores the sophisticated networking layer enabling Pod-to-Pod communication across nodes, Service discovery mechanisms, Ingress controllers for HTTP/HTTPS routing, and NetworkPolicies for micro-segmentation. You will understand CNI plugins (Calico, Cilium, Flannel), DNS resolution within the cluster, TLS termination patterns, and how the flat network model implements the Kubernetes requirement that all Pods communicate without NAT, forming the communication fabric upon which your Deployments and Services operate.